# 🏠 HomeMatch Tutorial: Personalized Real Estate Listings with LLMs + ChromaDB
**Welcome!** This notebook will guide you through building a fully working, AI-powered real estate personalization app — *HomeMatch*. We’ll use GPT-3.5 (via OpenAI API) and ChromaDB to semantically match property listings to user preferences and rewrite them with a personal touch.

## 🔧 Step 1: Install and Import Dependencies

In [1]:
%pip install --quiet langchain openai chromadb sentence-transformers ipywidgets transformers tiktoken

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import IPython

IPython.get_ipython().kernel.do_shutdown(restart=True)

{'status': 'ok', 'restart': True}

: 

In [1]:
import os, json
import openai
import ipywidgets as widgets
from IPython.display import display, Markdown
from sentence_transformers import SentenceTransformer
import chromadb
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

## 🔐 Step 2: Set OpenAI API Key (Vocareum)

In [ ]:
# Replace with your actual Vocareum API key
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = "<YOUR_API_KEY>"  # Replace with your actual API key
os.environ["OPENAI_API_KEY"] = openai.api_key  
os.environ["OPENAI_API_BASE"] = openai.api_base


## 🤖 Step 3: Define the GPT-3.5 LLM Client

In [3]:
class LLMClient:
    def __init__(self, temperature=0.7):
        self.llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=temperature)
    def ask(self, prompt):
        return self.llm([HumanMessage(content=prompt)]).content

## 🏡 Step 4: Generate Real Estate Listings

In [6]:
llm = LLMClient()
LISTINGS_PATH = "listings.json"
prompt = '''Generate exactly 20 diverse real estate property listings. Each listing must be formatted exactly like this:

Neighborhood: [name]
Price: [$amount]
Bedrooms: [number]
Bathrooms: [number]
House Size: [area in sqft]
Description: [3–5 sentence paragraph]
Neighborhood Description: [2–4 sentence paragraph]

One example is given to you for your reference:

Neighborhood: Green Oaks
Price: $800,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft
Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.
Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.

Separate each listing with a blank line. Output raw plain text only.'''

raw_text = llm.ask(prompt).strip()

# Save the generated listings to a text file
with open("Listings.txt", "w") as f:
    f.write(raw_text)
print("✅ Listings saved to Listings.txt")
print(raw_text[:1000])  # Preview first few lines
listings = []
blocks = raw_text.strip().split("\n\n")  # Each listing is a block

for block in blocks:
    lines = block.strip().split("\n")
    if len(lines) != 7:
        print(f"⚠️ Skipping invalid block:\n{block}\n")
        continue

    listing = {}

    for line in lines:
        key, value = line.split(":", 1)
        key = key.strip().lower().replace(" ", "_")  # Normalize
        value = value.strip()
        listing[key] = value

    listings.append(listing)

# Save as JSON
with open(LISTINGS_PATH, "w") as f:
    json.dump(listings, f, indent=2)

print(f"✅ Converted {len(listings)} listings and saved to {LISTINGS_PATH}")


✅ Listings saved to Listings.txt
Neighborhood: Downtown
Price: $1,200,000
Bedrooms: 2
Bathrooms: 2.5
House Size: 1,800 sqft
Description: Live in luxury at this sleek modern condo in the heart of Downtown. This 2-bedroom, 2.5-bathroom unit features high-end finishes, floor-to-ceiling windows with panoramic city views, and a gourmet kitchen with top-of-the-line appliances. Enjoy the convenience of walking to the trendiest restaurants, shops, and entertainment venues in the city. Experience urban living at its finest in this Downtown gem.
Neighborhood Description: Downtown is a bustling neighborhood with a vibrant nightlife, cultural attractions, and easy access to public transportation. Residents can enjoy a variety of dining options, art galleries, and theaters within walking distance. Experience the pulse of the city in this dynamic neighborhood.

Neighborhood: Lakeview
Price: $900,000
Bedrooms: 4
Bathrooms: 3
House Size: 2,500 sqft
Description: Welcome home to this charming 4-bedroom,

## 🧠 Step 5: Create ChromaDB and Embed Listings

In [11]:
db = chromadb.PersistentClient(path="chroma_store")
collection_name = "real_estate"
try:
    db.delete_collection(collection_name)
except:
    pass
col = db.get_or_create_collection(collection_name)
model = SentenceTransformer("all-MiniLM-L6-v2")
with open(LISTINGS_PATH) as f:
    listings = json.load(f)
for idx, listing in enumerate(listings):
    text = listing["neighborhood"] + "\n" + listing["house_size"] + "\n" + listing["description"] + "\n" + listing["neighborhood_description"]
    vec = model.encode(text).tolist()
    col.add(documents=[text], metadatas=[listing], ids=[f"listing-{idx}"], embeddings=[vec])

Failed to send telemetry event client_start: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event collection_add: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event collection_add: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event collection_add: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event collection_add: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event collection_add: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event collection_add: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event collection_add: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event collection_add: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event collection_add: capture() takes 1 positional argument

## 🧍 Step 6: Collect Buyer Preferences

In [ ]:
questions = [
    "How big do you want your house to be?",
    "What are 3 most important things for you in choosing this property? (Separate with commas)",
    "Which amenities would you like? (e.g., pool, gym, garden)",
    "Which transportation options are important to you? (e.g., bus, subway, car)",
    "How urban do you want your neighborhood to be? (Enter a number from 1 to 10, where 1 is rural and 10 is urban)",
]
inputs = [widgets.Textarea(placeholder=q) for q in questions]
submit = widgets.Button(description="Find Homes 🏡")
box = widgets.VBox(inputs + [submit])
display(box)
user_preferences = {}
def on_submit(b):
    for i, input_box in enumerate(inputs):
        user_preferences[questions[i]] = input_box.value
    display(Markdown("### ✅ Preferences captured. Proceed to search."))
submit.on_click(on_submit)

### ✅ Preferences captured. Proceed to search.

## 🔍 Step 7: Semantic Search for Matching Listings

In [13]:
# Run this AFTER submitting preferences
if user_preferences:
    query_text = " ".join(user_preferences.values())
    vec = model.encode(query_text).tolist()
    results = col.query(query_embeddings=[vec], n_results=5, include=["metadatas"])
    matched = results["metadatas"][0]

## ✍️ Step 8: Personalize Listings with GPT

In [14]:
personalized = []
for listing in matched:
    prompt = f'''
You are a real estate agent personalizing listings.
Buyer's Preferences:
{query_text}

Original Listing:
{listing['description']}

Neighborhood:
{listing['neighborhood_description']}

Rewrite the listing to emphasize the buyer's preferences.
'''
    new_text = llm.ask(prompt)
    personalized.append((listing, new_text))

## 🖨️ Step 9: Display Personalized Listings

In [15]:
for idx, (original, personalized_text) in enumerate(personalized, 1):
    display(Markdown(f"### **🏘️ Listing {idx}**"))
    display(Markdown(f"**About the Property:**\n\n{personalized_text}"))
    display(Markdown("**Details:**"))
    for key in ["neighborhood", "price", "bedrooms", "bathrooms", "house_size"]:
        display(Markdown(f"**{key.capitalize().replace('_', ' ')}:** {original.get(key, 'N/A')}"))

### **🏘️ Listing 1**

**About the Property:**

Live in luxury and enjoy resort-style amenities at this stunning 2500 sqft condo in a semi-urban setting. This 2-bedroom, 2.5-bathroom unit boasts a swimming pool, play area, community gym, sports facilities, and a library. With floor-to-ceiling windows offering panoramic views, a gourmet kitchen with top-of-the-line appliances, and easy access to bus and train stations, this condo is perfect for those seeking convenience and a vibrant lifestyle. Experience the best of both worlds in this sleek modern condo located in a dynamic neighborhood with endless entertainment options.

**Details:**

**Neighborhood:** Downtown

**Price:** $1,200,000

...

**Neighborhood:** Downtown East

**Price:** $750,000

**Bedrooms:** 3

**Bathrooms:** 2

**House size:** 1,800 sqft